# Dataset Scratch

In [1]:
## imports
import torch
from torch.utils.data import DataLoader  # Minibatches
import torchvision.datasets as datasets  # MNIST dataset
import torchvision.transforms as transforms
import numpy as np
import torch.nn as nn   # Nueral network modules.
from collections import OrderedDict

In [2]:
class NN(nn.Module):

    def __init__(self, input_size, middle_width, num_classes):


        super(NN, self).__init__()
        self.features = nn.Sequential(OrderedDict([
            ('hidden_layer', nn.Linear(input_size, middle_width)),
            ('hidden_activation', nn.ReLU()),
            ('readout', nn.Linear(middle_width, num_classes))
        ]))

    def forward(self, x):
        x = self.features(x)

        return x

Setting Device

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

Mnist Dataset

In [4]:
dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)

In [5]:
dataset

Dataset MNIST
    Number of datapoints: 60000
    Root location: dataset/
    Split: Train
    StandardTransform
Transform: ToTensor()

MNIST Dataset Function

In [6]:
def mnist_dataset(batch_size, train=True, values=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]):
    dataset = datasets.MNIST(root='dataset/', train=train, transform=transforms.ToTensor(), download=True)
    list = []
    targets_list = dataset.targets.tolist()
    for i in range(len(dataset)):
        if targets_list[i] in values:
            list.append(i)
    dataset1 = torch.utils.data.Subset(dataset, list)
    loader = DataLoader(dataset=dataset1, batch_size=batch_size, shuffle=True)
    return loader

MNIST Subset Class

In [7]:
## extend from torch dataset.

class CustomDataset:
    def __init__(self, data, target):
        self.data = data
        self.targets = target
        
    def __len__(self):
        return self.data
    
    def __getitem__(self, idx):
        current_sample = self.data[idx, :]
        current_target = self.targets[idx]
        return {
            "sample":torch.tensor(current_sample, dtype=torch.float),
            "target": torch.tensor(current_target, dtype=torch.long)
        }

In [44]:
def check_accuracy(device, model, loader):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)
            y = classify(y, [2, 7])
            x = x.reshape(x.shape[0], -1)
            

            scores = model(x)
            # 64images x 10,

            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    return num_correct / num_samples


Function that changes the clasification targets of MNSIT values

In [9]:
def classify(targets, values):
    new_targets = targets.clone()
    for key, element in enumerate(values):
        new_targets[new_targets == element] = key
    return new_targets

TEST

In [10]:
model = NN(input_size=784,
           middle_width=100,
            num_classes=2).to(device=device)

In [11]:
# Loading dataset
loader = mnist_dataset(100, values=[0, 1])

In [12]:
loss_function = nn.CrossEntropyLoss()

In [13]:
# Getting datapoints for training (but not training)
for batch_idx, (data, targets) in enumerate(loader):
    data = data.reshape(data.shape[0], -1).to(device=device)
    targets = targets.to(device=device)
    scores = model(data)
    loss = loss_function(scores, classify(targets, [0, 1]))

In [14]:
targets

tensor([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1,
        0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1,
        0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1])

In [37]:
classify(targets, [0, 1])

ValueError: too many values to unpack (expected 2)

In [45]:
ratio =  check_accuracy(device, model, loader)

In [47]:
100 - 100 * ratio

tensor(23.5926)

In [42]:
right

12665

In [43]:
samples/right

tensor(0.7641)

In [34]:
scores

tensor([[ 0.1207,  0.1467],
        [-0.0294, -0.0953],
        [-0.0160, -0.0604],
        [-0.1551, -0.0469],
        [-0.1030,  0.1157],
        [-0.0036, -0.0198],
        [-0.0681, -0.0195],
        [ 0.0025,  0.0543],
        [-0.1050,  0.0438],
        [ 0.1190,  0.0709],
        [-0.0843,  0.0149],
        [-0.0993,  0.0287],
        [ 0.0951, -0.0436],
        [ 0.0108,  0.0435],
        [-0.0550, -0.0384],
        [-0.1378,  0.0780],
        [-0.0483, -0.0033],
        [-0.1152,  0.0734],
        [-0.0853,  0.0979],
        [-0.1333, -0.1260],
        [ 0.0470,  0.0040],
        [-0.1152,  0.0445],
        [-0.1524,  0.0414],
        [-0.0766, -0.0531],
        [-0.0124,  0.0145],
        [-0.1001,  0.0462],
        [-0.1090,  0.0814],
        [-0.1192,  0.0060],
        [-0.0058,  0.0477],
        [-0.0339,  0.0125],
        [-0.0181, -0.0659],
        [-0.0470,  0.0378],
        [-0.0604,  0.0243],
        [ 0.0215,  0.0294],
        [ 0.1612,  0.0003],
        [-0.0731,  0

In [33]:
scores.max(1)

torch.return_types.max(
values=tensor([ 0.1467, -0.0294, -0.0160, -0.0469,  0.1157, -0.0036, -0.0195,  0.0543,
         0.0438,  0.1190,  0.0149,  0.0287,  0.0951,  0.0435, -0.0384,  0.0780,
        -0.0033,  0.0734,  0.0979, -0.1260,  0.0470,  0.0445,  0.0414, -0.0531,
         0.0145,  0.0462,  0.0814,  0.0060,  0.0477,  0.0125, -0.0181,  0.0378,
         0.0243,  0.0294,  0.1612,  0.0404,  0.0823,  0.1220,  0.0949,  0.0652,
         0.0578,  0.1078,  0.1071, -0.0039,  0.0496,  0.0937,  0.0734,  0.0136,
         0.0694, -0.0767, -0.0272, -0.0120, -0.0162, -0.0482,  0.0438,  0.0673,
         0.1071,  0.0769,  0.0658,  0.1050, -0.0003,  0.0190,  0.0200,  0.0578,
         0.0894]),
indices=tensor([1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]))

In [21]:
scores.max(1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
y = check_accuracy(device, model, loader)

In [24]:
values = [0, 1, 2]

In [25]:
len(values)

3

In [22]:
_, predictions = scores.max(1)

IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [26]:
scores

tensor([0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0,
        1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1,
        1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0])

In [ ]:
predictions

In [ ]:
(predictions == y).sum()

In [ ]:
100 - 100*float((predictions==y).sum())/float(y.size(0))